In [1]:
# data manipulation
import numpy as np
import pandas as pd

#file processing / requests
from pathlib import Path
from tqdm import tqdm
import json
import urllib

#computer vision
import cv2
import torch
import torchvision

import PIL

#ML 
from sklearn.model_selection import train_test_split


#display
from IPython.display import display
from pylab import rcParams

#visualization
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
import PIL.Image as Image

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', None)

### Notebook Purpose: Implement Background Subtraction using OpenCV C++ and Python methods

In [41]:
!pip install hdbscan

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cached Cython-0.29.27-py2.py3-none-any.whl (983 kB)
  Created wheel for hdbscan: filename=hdbscan-0.8.27-cp36-cp36m-win_amd64.whl size=634049 sha256=80442fa90ee8e469a2577a7f896aa984e8293c922c748f65a954690cd312913e
  Stored in directory: c:\users\phil0\appdata\local\pip\cache\wheels\70\52\fe\83ef5a0e879c48df9968d931d41046226e4cb19a166f0223d2
Successfully built hdbscan


In [64]:
ALGORITHMS_TO_EVALUATE = [
    (cv.bgsegm.createBackgroundSubtractorMOG, 'MOG', {}),
    (cv.bgsegm.createBackgroundSubtractorGMG, 'GMG', {}),
    (cv.bgsegm.createBackgroundSubtractorCNT, 'CNT', {}),
    (cv.bgsegm.createBackgroundSubtractorLSBP, 'LSBP-vanilla', {'nSamples': 20, 'LSBPRadius': 4, 'Tlower': 2.0, 'Tupper': 200.0, 'Tinc': 1.0, 'Tdec': 0.05, 'Rscale': 5.0, 'Rincdec': 0.05, 'LSBPthreshold': 8}),
    (cv.bgsegm.createBackgroundSubtractorLSBP, 'LSBP-speed', {'nSamples': 10, 'LSBPRadius': 16, 'Tlower': 2.0, 'Tupper': 32.0, 'Tinc': 1.0, 'Tdec': 0.05, 'Rscale': 10.0, 'Rincdec': 0.005, 'LSBPthreshold': 8}),
    (cv.bgsegm.createBackgroundSubtractorLSBP, 'LSBP-quality', {'nSamples': 20, 'LSBPRadius': 16, 'Tlower': 2.0, 'Tupper': 32.0, 'Tinc': 1.0, 'Tdec': 0.05, 'Rscale': 10.0, 'Rincdec': 0.005, 'LSBPthreshold': 8}),
    (cv.bgsegm.createBackgroundSubtractorLSBP, 'LSBP-camera-motion-compensation', {'mc': 1}),
    (cv.bgsegm.createBackgroundSubtractorGSOC, 'GSOC', {}),
    (cv.bgsegm.createBackgroundSubtractorGSOC, 'GSOC-camera-motion-compensation', {'mc': 1})
]

algos_names = [algo[1] for algo in ALGORITHMS_TO_EVALUATE]
algos_names

NameError: name 'cv' is not defined

In [71]:
import typing
from typing import List

algo_names = ['MOG','GMG','CNT','LSBP-vanilla','LSBP-speed','LSBP-quality','LSBP-camera-motion-compensation','GSOC','GSOC-camera-motion-compensation']

def generate_background_subtraction_commands(algos: List):
    for algo in algos:
        print(f'!python background_subtraction_evaluation.py --algorithm {algo} --dataset_path /Users/phil0/DS/Video_Background_Subtraction_OpenCV/cdnet14_baseline_dataset/baseline/highway')
generate_background_subtraction_commands(algo_names)    

!python background_subtraction_evaluation.py --algorithm MOG --dataset_path /Users/phil0/DS/Video_Background_Subtraction_OpenCV/cdnet14_baseline_dataset/baseline/highway
!python background_subtraction_evaluation.py --algorithm GMG --dataset_path /Users/phil0/DS/Video_Background_Subtraction_OpenCV/cdnet14_baseline_dataset/baseline/highway
!python background_subtraction_evaluation.py --algorithm CNT --dataset_path /Users/phil0/DS/Video_Background_Subtraction_OpenCV/cdnet14_baseline_dataset/baseline/highway
!python background_subtraction_evaluation.py --algorithm LSBP-vanilla --dataset_path /Users/phil0/DS/Video_Background_Subtraction_OpenCV/cdnet14_baseline_dataset/baseline/highway
!python background_subtraction_evaluation.py --algorithm LSBP-speed --dataset_path /Users/phil0/DS/Video_Background_Subtraction_OpenCV/cdnet14_baseline_dataset/baseline/highway
!python background_subtraction_evaluation.py --algorithm LSBP-quality --dataset_path /Users/phil0/DS/Video_Background_Subtraction_Open

### Baseline

In [87]:
!python background_subtraction_evaluation.py --algorithm MOG --dataset_path /Users/phil0/DS/Video_Background_Subtraction_OpenCV/cdnet14_baseline_dataset/baseline/highway

0.0 sec to complete <function get_files_in_dir at 0x0000028A2207F950>
0.0 sec to complete <function get_files_in_dir at 0x0000028A2207F950>
=== baseline:highway ===
Algorithm name: MOG
bgs <bgsegm_BackgroundSubtractorMOG 0000028A22C6EF10>
Average accuracy: 0.964
Average precision: 0.869
Average recall: 0.354
Average F1: 0.492
Average sec. per step: 0.0017

0.0 sec to complete <function get_files_in_dir at 0x0000028A2207F950>
0.0 sec to complete <function get_files_in_dir at 0x0000028A2207F950>
=== baseline:highway ===
Algorithm name: MOG
bgs <bgsegm_BackgroundSubtractorMOG 0000028A22C6EF30>
Average accuracy: 0.964
Average precision: 0.869
Average recall: 0.354
Average F1: 0.492
Average sec. per step: 0.0017

=== SUMMARY for baseline (Precision, Recall, F1, Accuracy) ===
  MOG: 0.869 0.354 0.492 0.964


In [89]:
!python background_subtraction_evaluation.py --algorithm GMG --dataset_path /Users/phil0/DS/Video_Background_Subtraction_OpenCV/cdnet14_baseline_dataset/baseline/highway

0.0 sec to complete <function get_files_in_dir at 0x0000025FE7B81950>
0.0 sec to complete <function get_files_in_dir at 0x0000025FE7B81950>
=== baseline:highway ===
Algorithm name: GMG
bgs <bgsegm_BackgroundSubtractorGMG 0000025FE87AAAF0>
Average accuracy: 0.994
Average precision: 0.915
Average recall: 0.995
Average F1: 0.952
Average sec. per step: 0.0058

0.0 sec to complete <function get_files_in_dir at 0x0000025FE7B81950>
0.0 sec to complete <function get_files_in_dir at 0x0000025FE7B81950>
=== baseline:highway ===
Algorithm name: GMG
bgs <bgsegm_BackgroundSubtractorGMG 0000025FE87AAB10>
Average accuracy: 0.994
Average precision: 0.915
Average recall: 0.995
Average F1: 0.952
Average sec. per step: 0.0056

=== SUMMARY for baseline (Precision, Recall, F1, Accuracy) ===
  GMG: 0.915 0.995 0.952 0.994


In [90]:
!python background_subtraction_evaluation.py --algorithm LSBP-vanilla --dataset_path /Users/phil0/DS/Video_Background_Subtraction_OpenCV/cdnet14_baseline_dataset/baseline/highway

0.0 sec to complete <function get_files_in_dir at 0x000001D069860950>
0.0 sec to complete <function get_files_in_dir at 0x000001D069860950>
=== baseline:highway ===
Algorithm name: LSBP-vanilla
bgs <bgsegm_BackgroundSubtractorLSBP 000001D06A4A0970>
Average accuracy: 0.764
Average precision: 0.120
Average recall: 0.593
Average F1: 0.185
Average sec. per step: 0.0054

0.0 sec to complete <function get_files_in_dir at 0x000001D069860950>
0.0 sec to complete <function get_files_in_dir at 0x000001D069860950>
=== baseline:highway ===
Algorithm name: LSBP-vanilla
bgs <bgsegm_BackgroundSubtractorLSBP 000001D06A4A0990>
Average accuracy: 0.764
Average precision: 0.120
Average recall: 0.593
Average F1: 0.185
Average sec. per step: 0.0055

=== SUMMARY for baseline (Precision, Recall, F1, Accuracy) ===
LSBP-vanilla: 0.120 0.593 0.185 0.764


In [91]:
!python background_subtraction_evaluation.py --algorithm LSBP-speed --dataset_path /Users/phil0/DS/Video_Background_Subtraction_OpenCV/cdnet14_baseline_dataset/baseline/highway

0.0 sec to complete <function get_files_in_dir at 0x0000023529BAF950>
0.0 sec to complete <function get_files_in_dir at 0x0000023529BAF950>
=== baseline:highway ===
Algorithm name: LSBP-speed
bgs <bgsegm_BackgroundSubtractorLSBP 000002352A83B150>
Average accuracy: 0.849
Average precision: 0.253
Average recall: 0.967
Average F1: 0.385
Average sec. per step: 0.0055

0.0 sec to complete <function get_files_in_dir at 0x0000023529BAF950>
0.0 sec to complete <function get_files_in_dir at 0x0000023529BAF950>
=== baseline:highway ===
Algorithm name: LSBP-speed
bgs <bgsegm_BackgroundSubtractorLSBP 000002352A83B170>
Average accuracy: 0.849
Average precision: 0.253
Average recall: 0.965
Average F1: 0.384
Average sec. per step: 0.0056

=== SUMMARY for baseline (Precision, Recall, F1, Accuracy) ===
LSBP-speed: 0.253 0.966 0.384 0.849


In [92]:
!python background_subtraction_evaluation.py --algorithm LSBP-quality --dataset_path /Users/phil0/DS/Video_Background_Subtraction_OpenCV/cdnet14_baseline_dataset/baseline/highway

0.0 sec to complete <function get_files_in_dir at 0x0000019E8F32F950>
0.0 sec to complete <function get_files_in_dir at 0x0000019E8F32F950>
=== baseline:highway ===
Algorithm name: LSBP-quality
bgs <bgsegm_BackgroundSubtractorLSBP 0000019E8FF47DF0>
Average accuracy: 0.875
Average precision: 0.277
Average recall: 0.859
Average F1: 0.399
Average sec. per step: 0.0057

0.0 sec to complete <function get_files_in_dir at 0x0000019E8F32F950>
0.0 sec to complete <function get_files_in_dir at 0x0000019E8F32F950>
=== baseline:highway ===
Algorithm name: LSBP-quality
bgs <bgsegm_BackgroundSubtractorLSBP 0000019E8FF47E10>
Average accuracy: 0.874
Average precision: 0.276
Average recall: 0.855
Average F1: 0.398
Average sec. per step: 0.0058

=== SUMMARY for baseline (Precision, Recall, F1, Accuracy) ===
LSBP-quality: 0.277 0.857 0.399 0.875


In [93]:
!python background_subtraction_evaluation.py --algorithm LSBP-camera-motion-compensation --dataset_path /Users/phil0/DS/Video_Background_Subtraction_OpenCV/cdnet14_baseline_dataset/baseline/highway

0.0 sec to complete <function get_files_in_dir at 0x000001BE961DF950>
0.0 sec to complete <function get_files_in_dir at 0x000001BE961DF950>
=== baseline:highway ===
Algorithm name: LSBP-camera-motion-compensation
bgs <bgsegm_BackgroundSubtractorLSBP 000001BE96DFAD10>
Average accuracy: 0.875
Average precision: 0.276
Average recall: 0.856
Average F1: 0.398
Average sec. per step: 0.0135

0.0 sec to complete <function get_files_in_dir at 0x000001BE961DF950>
0.0 sec to complete <function get_files_in_dir at 0x000001BE961DF950>
=== baseline:highway ===
Algorithm name: LSBP-camera-motion-compensation
bgs <bgsegm_BackgroundSubtractorLSBP 000001BE96DFAD30>
Average accuracy: 0.874
Average precision: 0.276
Average recall: 0.855
Average F1: 0.398
Average sec. per step: 0.0135

=== SUMMARY for baseline (Precision, Recall, F1, Accuracy) ===
LSBP-camera-motion-compensation: 0.276 0.855 0.398 0.874


In [94]:
!python background_subtraction_evaluation.py --algorithm GSOC --dataset_path /Users/phil0/DS/Video_Background_Subtraction_OpenCV/cdnet14_baseline_dataset/baseline/highway

0.0 sec to complete <function get_files_in_dir at 0x000001FB4464F950>
0.0 sec to complete <function get_files_in_dir at 0x000001FB4464F950>
=== baseline:highway ===
Algorithm name: GSOC
bgs <bgsegm_BackgroundSubtractorGSOC 000001FB452B5790>
Average accuracy: 0.998
Average precision: 0.998
Average recall: 0.964
Average F1: 0.980
Average sec. per step: 0.0040

0.0 sec to complete <function get_files_in_dir at 0x000001FB4464F950>
0.0 sec to complete <function get_files_in_dir at 0x000001FB4464F950>
=== baseline:highway ===
Algorithm name: GSOC
bgs <bgsegm_BackgroundSubtractorGSOC 000001FB452B57B0>
Average accuracy: 0.998
Average precision: 0.998
Average recall: 0.964
Average F1: 0.980
Average sec. per step: 0.0041

=== SUMMARY for baseline (Precision, Recall, F1, Accuracy) ===
 GSOC: 0.998 0.964 0.980 0.998


In [95]:
!python background_subtraction_evaluation.py --algorithm GSOC-camera-motion-compensation --dataset_path /Users/phil0/DS/Video_Background_Subtraction_OpenCV/cdnet14_baseline_dataset/baseline/highway

0.0 sec to complete <function get_files_in_dir at 0x0000024B630DF950>
0.0 sec to complete <function get_files_in_dir at 0x0000024B630DF950>
=== baseline:highway ===
Algorithm name: GSOC-camera-motion-compensation
bgs <bgsegm_BackgroundSubtractorGSOC 0000024B63CF7DF0>
Average accuracy: 0.998
Average precision: 0.997
Average recall: 0.965
Average F1: 0.980
Average sec. per step: 0.0129

0.0 sec to complete <function get_files_in_dir at 0x0000024B630DF950>
0.0 sec to complete <function get_files_in_dir at 0x0000024B630DF950>
=== baseline:highway ===
Algorithm name: GSOC-camera-motion-compensation
bgs <bgsegm_BackgroundSubtractorGSOC 0000024B63CF7E10>
Average accuracy: 0.998
Average precision: 0.998
Average recall: 0.964
Average F1: 0.980
Average sec. per step: 0.0132

=== SUMMARY for baseline (Precision, Recall, F1, Accuracy) ===
GSOC-camera-motion-compensation: 0.998 0.965 0.980 0.998


### Shadows

In [98]:
!python background_subtraction_evaluation.py --algorithm MOG --dataset_path /Users/phil0/DS/Video_Background_Subtraction_OpenCV/cdnet_14_dataset/dataset/shadow/cubicle 

0.0 sec to complete <function get_files_in_dir at 0x000002208B59F950>
0.0 sec to complete <function get_files_in_dir at 0x000002208B59F950>
=== shadow:cubicle ===
Algorithm name: MOG
bgs <bgsegm_BackgroundSubtractorMOG 000002208C1B8DB0>
Average accuracy: 0.990
Average precision: 0.662
Average recall: 0.465
Average F1: 0.414
Average sec. per step: 0.0019

0.0 sec to complete <function get_files_in_dir at 0x000002208B59F950>
0.0 sec to complete <function get_files_in_dir at 0x000002208B59F950>
=== shadow:cubicle ===
Algorithm name: MOG
bgs <bgsegm_BackgroundSubtractorMOG 000002208C1B8DB0>
Average accuracy: 0.990
Average precision: 0.662
Average recall: 0.465
Average F1: 0.414
Average sec. per step: 0.0019

=== SUMMARY for shadow (Precision, Recall, F1, Accuracy) ===
  MOG: 0.662 0.465 0.414 0.990


In [99]:
!python background_subtraction_evaluation.py --algorithm GSOC-camera-motion-compensation --dataset_path /Users/phil0/DS/Video_Background_Subtraction_OpenCV/cdnet_14_dataset/dataset/shadow/cubicle

0.0 sec to complete <function get_files_in_dir at 0x00000278DDD5F950>
0.0 sec to complete <function get_files_in_dir at 0x00000278DDD5F950>
=== shadow:cubicle ===
Algorithm name: GSOC-camera-motion-compensation
bgs <bgsegm_BackgroundSubtractorGSOC 00000278DE968EF0>
Average accuracy: 0.989
Average precision: 0.499
Average recall: 0.814
Average F1: 0.496
Average sec. per step: 0.0151

0.0 sec to complete <function get_files_in_dir at 0x00000278DDD5F950>
0.0 sec to complete <function get_files_in_dir at 0x00000278DDD5F950>
=== shadow:cubicle ===
Algorithm name: GSOC-camera-motion-compensation
bgs <bgsegm_BackgroundSubtractorGSOC 00000278DD0F6A90>
Average accuracy: 0.989
Average precision: 0.494
Average recall: 0.815
Average F1: 0.492
Average sec. per step: 0.0147

=== SUMMARY for shadow (Precision, Recall, F1, Accuracy) ===
GSOC-camera-motion-compensation: 0.497 0.814 0.494 0.989


In [100]:
!python background_subtraction_evaluation.py --algorithm GSOC --dataset_path /Users/phil0/DS/Video_Background_Subtraction_OpenCV/cdnet_14_dataset/dataset/shadow/cubicle

0.0 sec to complete <function get_files_in_dir at 0x000001819651F950>
0.0 sec to complete <function get_files_in_dir at 0x000001819651F950>
=== shadow:cubicle ===
Algorithm name: GSOC
bgs <bgsegm_BackgroundSubtractorGSOC 00000181971619D0>
Average accuracy: 0.991
Average precision: 0.520
Average recall: 0.814
Average F1: 0.513
Average sec. per step: 0.0047

0.0 sec to complete <function get_files_in_dir at 0x000001819651F950>
0.0 sec to complete <function get_files_in_dir at 0x000001819651F950>
=== shadow:cubicle ===
Algorithm name: GSOC
bgs <bgsegm_BackgroundSubtractorGSOC 00000181971619D0>
Average accuracy: 0.991
Average precision: 0.523
Average recall: 0.815
Average F1: 0.514
Average sec. per step: 0.0045

=== SUMMARY for shadow (Precision, Recall, F1, Accuracy) ===
 GSOC: 0.522 0.814 0.513 0.991


In [101]:
!python background_subtraction_evaluation.py --algorithm LSBP-camera-motion-compensation --dataset_path /Users/phil0/DS/Video_Background_Subtraction_OpenCV/cdnet_14_dataset/dataset/shadow/cubicle

0.0 sec to complete <function get_files_in_dir at 0x000002CCB5F0F950>
0.0 sec to complete <function get_files_in_dir at 0x000002CCB5F0F950>
=== shadow:cubicle ===
Algorithm name: LSBP-camera-motion-compensation
bgs <bgsegm_BackgroundSubtractorLSBP 000002CCB6BAA0F0>
Average accuracy: 0.949
Average precision: 0.172
Average recall: 0.963
Average F1: 0.236
Average sec. per step: 0.0156

0.0 sec to complete <function get_files_in_dir at 0x000002CCB5F0F950>
0.0 sec to complete <function get_files_in_dir at 0x000002CCB5F0F950>
=== shadow:cubicle ===
Algorithm name: LSBP-camera-motion-compensation
bgs <bgsegm_BackgroundSubtractorLSBP 000002CCB6BAA0F0>
Average accuracy: 0.950
Average precision: 0.173
Average recall: 0.965
Average F1: 0.237
Average sec. per step: 0.0149

=== SUMMARY for shadow (Precision, Recall, F1, Accuracy) ===
LSBP-camera-motion-compensation: 0.172 0.964 0.237 0.949


In [102]:
!python background_subtraction_evaluation.py --algorithm LSBP-quality --dataset_path /Users/phil0/DS/Video_Background_Subtraction_OpenCV/cdnet_14_dataset/dataset/shadow/cubicle

0.0 sec to complete <function get_files_in_dir at 0x000002408B59F950>
0.0 sec to complete <function get_files_in_dir at 0x000002408B59F950>
=== shadow:cubicle ===
Algorithm name: LSBP-quality
bgs <bgsegm_BackgroundSubtractorLSBP 000002408C220310>
Average accuracy: 0.959
Average precision: 0.192
Average recall: 0.965
Average F1: 0.254
Average sec. per step: 0.0063

0.0 sec to complete <function get_files_in_dir at 0x000002408B59F950>
0.0 sec to complete <function get_files_in_dir at 0x000002408B59F950>
=== shadow:cubicle ===
Algorithm name: LSBP-quality
bgs <bgsegm_BackgroundSubtractorLSBP 000002408C220310>
Average accuracy: 0.959
Average precision: 0.190
Average recall: 0.965
Average F1: 0.252
Average sec. per step: 0.0064

=== SUMMARY for shadow (Precision, Recall, F1, Accuracy) ===
LSBP-quality: 0.191 0.965 0.253 0.959


In [103]:
!python background_subtraction_evaluation.py --algorithm LSBP-speed --dataset_path /Users/phil0/DS/Video_Background_Subtraction_OpenCV/cdnet_14_dataset/dataset/shadow/cubicle

0.0 sec to complete <function get_files_in_dir at 0x000001CE0B59F950>
0.0 sec to complete <function get_files_in_dir at 0x000001CE0B59F950>
=== shadow:cubicle ===
Algorithm name: LSBP-speed
bgs <bgsegm_BackgroundSubtractorLSBP 000001CE0C1B8D90>
Average accuracy: 0.954
Average precision: 0.175
Average recall: 0.975
Average F1: 0.236
Average sec. per step: 0.0061

0.0 sec to complete <function get_files_in_dir at 0x000001CE0B59F950>
0.0 sec to complete <function get_files_in_dir at 0x000001CE0B59F950>
=== shadow:cubicle ===
Algorithm name: LSBP-speed
bgs <bgsegm_BackgroundSubtractorLSBP 000001CE0C1B8D90>
Average accuracy: 0.953
Average precision: 0.174
Average recall: 0.975
Average F1: 0.236
Average sec. per step: 0.0059

=== SUMMARY for shadow (Precision, Recall, F1, Accuracy) ===
LSBP-speed: 0.174 0.975 0.236 0.954


In [104]:
!python background_subtraction_evaluation.py --algorithm LSBP-vanilla --dataset_path /Users/phil0/DS/Video_Background_Subtraction_OpenCV/cdnet_14_dataset/dataset/shadow/cubicle

0.0 sec to complete <function get_files_in_dir at 0x00000213EB45F950>
0.0 sec to complete <function get_files_in_dir at 0x00000213EB45F950>
=== shadow:cubicle ===
Algorithm name: LSBP-vanilla
bgs <bgsegm_BackgroundSubtractorLSBP 00000213EC08AB10>
Average accuracy: 0.931
Average precision: 0.168
Average recall: 0.898
Average F1: 0.222
Average sec. per step: 0.0061

0.0 sec to complete <function get_files_in_dir at 0x00000213EB45F950>
0.0 sec to complete <function get_files_in_dir at 0x00000213EB45F950>
=== shadow:cubicle ===
Algorithm name: LSBP-vanilla
bgs <bgsegm_BackgroundSubtractorLSBP 00000213EC08AB10>
Average accuracy: 0.931
Average precision: 0.167
Average recall: 0.897
Average F1: 0.221
Average sec. per step: 0.0062

=== SUMMARY for shadow (Precision, Recall, F1, Accuracy) ===
LSBP-vanilla: 0.167 0.897 0.221 0.931


In [105]:
!python background_subtraction_evaluation.py --algorithm GMG --dataset_path /Users/phil0/DS/Video_Background_Subtraction_OpenCV/cdnet_14_dataset/dataset/shadow/cubicle

0.0 sec to complete <function get_files_in_dir at 0x000002776D12F950>
0.0 sec to complete <function get_files_in_dir at 0x000002776D12F950>
=== shadow:cubicle ===
Algorithm name: GMG
bgs <bgsegm_BackgroundSubtractorGMG 000002776DDAC3D0>
Average accuracy: 0.984
Average precision: 0.302
Average recall: 0.541
Average F1: 0.280
Average sec. per step: 0.0049

0.0 sec to complete <function get_files_in_dir at 0x000002776D12F950>
0.0 sec to complete <function get_files_in_dir at 0x000002776D12F950>
=== shadow:cubicle ===
Algorithm name: GMG
bgs <bgsegm_BackgroundSubtractorGMG 000002776DDAC3D0>
Average accuracy: 0.984
Average precision: 0.302
Average recall: 0.541
Average F1: 0.280
Average sec. per step: 0.0050

=== SUMMARY for shadow (Precision, Recall, F1, Accuracy) ===
  GMG: 0.302 0.541 0.280 0.984


In [117]:
import sys
sys.path.append('/Users/Administrator/DS/DeepRoad/Object_Detection')
sys.path.append('/Users/phil0/DS/DeepRoad/Object_Detection')
import utils2
from importlib import reload
reload(utils2)
from utils2 import *

path1 = r'C:\Users\phil0\DS\Video_Background_Subtraction_OpenCV\cdnet_14_dataset\dataset\shadow\cubicle\input'
path_bs2fs(path1) 

'C:/Users/phil0/DS/Video_Background_Subtraction_OpenCV/cdnet_14_dataset/dataset/shadow/cubicle/input'


In [114]:
### Creating videos from our images

create_video_from_images('jpg', image_dir = 'C:/Users/phil0/DS/Video_Background_Subtraction_OpenCV/cdnet_14_dataset/dataset/shadow/cubicle/input', output_video_name = 'shadows_cubicle.mp4')

0.0 sec to complete <function get_files_in_dir at 0x0000023A34044730>
5.1 sec to complete <function create_video_from_images at 0x0000023A35DBB9D8>


In [119]:
path_highway = r'C:\Users\phil0\DS\Video_Background_Subtraction_OpenCV\cdnet14_baseline_dataset\baseline\highway\input'
path_highway_updated = path_bs2fs(path_highway) 

'C:/Users/phil0/DS/Video_Background_Subtraction_OpenCV/cdnet14_baseline_dataset/baseline/highway/input'


In [121]:
create_video_from_images('jpg', image_dir = r'C:/Users/phil0/DS/Video_Background_Subtraction_OpenCV/cdnet14_baseline_dataset/baseline/highway/input', output_video_name = 'baseline_highway.mp4', fps = 30)

0.0 sec to complete <function get_files_in_dir at 0x0000023A35E1CE18>
1.4 sec to complete <function create_video_from_images at 0x0000023A35DE6378>
